# Extract SrcTables Results in  OGA

 work with Weakly_2024_07
- use jupyter kernel LSST
- author : Sylvie Dagoret-Campagne
- affiliation : IJCLab
- creation date : 2024/02/18
- update : 2024/02/23


Try to find raw sources


In [1]:
! eups list -s | grep LOCAL

analysis_tools        LOCAL:/home/d/dagoret/repos/repos_w_2024_07/analysis_tools 	setup
atmospec              LOCAL:/home/d/dagoret/repos/repos_w_2024_07/atmospec 	setup
eups                  LOCAL:/opt/lsst/software/stack/conda/miniconda3-py38_4.9.2/envs/lsst-scipipe-8.0.0/eups 	setup
spectractor           LOCAL:/home/d/dagoret/repos/repos_w_2024_07/Spectractor 	setup


In [2]:
! eups list -s lsst_distrib

   g4213664e8e+34187316c4 	current w_2024_07 setup


In [3]:
import lsst.daf.butler as dafButler
#import lsst.summit.utils.butlerUtils as butlerUtils

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LogNorm,SymLogNorm


import matplotlib.ticker                         # here's where the formatter is
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.io import fits
from astropy.table import Table
import pandas as pd

import matplotlib.ticker                         # here's where the formatter is
import os
import re
import pandas as pd

plt.rcParams["figure.figsize"] = (4,3)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'

In [5]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter,
                               AutoMinorLocator)

from astropy.visualization import (MinMaxInterval, SqrtStretch,ZScaleInterval,PercentileInterval,
                                   ImageNormalize,imshow_norm)
from astropy.visualization.stretch import SinhStretch, LinearStretch,AsinhStretch,LogStretch

from astropy.time import Time


In [6]:
# LSST Display
import lsst.afw.display as afwDisplay
afwDisplay.setDefaultBackend('matplotlib')

In [7]:
transform = AsinhStretch() + PercentileInterval(99.)

In [8]:
pd.options.display.max_columns = None
#pd.options.display.max_rows = None



## Configuration

In [9]:
DATE = 202402
#filesource_out="finalized_src_table_202402.csv"
#filetractpatch_out = 'Latiss_tractspatchs_202312.pickle'

In [10]:
#repo =  "/sdf/group/rubin/repo/main"
repo = "/sdf/group/rubin/repo/oga/"
butler = dafButler.Butler(repo)
registry = butler.registry

In [11]:
#file_out="sourceTable_202312.csv"

## Give the collection

In [12]:
#my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_2023-11A-10A-09AB-08ABC-07AB-05AB/w_2023_46/PREOPS-4553"
my_collection = "LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871"

## Config

# Sources

In [13]:
# src : a Source catalog
# srcMatchFull
# finalized_src_table : sape of a pandas dataframe
data_product = "finalized_src_table" 
datasetRefs = butler.registry.queryDatasets(datasetType=data_product, collections=my_collection, where= "instrument='LATISS'")
filesource_out=f"{data_product}_{DATE}.csv"

In [ ]:
all_ref_for_src = []
all_dataId = []
all_src = []
all_exposures = []
all_sourcesTables = []
all_dateobs = []

for i, ref in enumerate(datasetRefs):
    all_ref_for_src.append(ref.dataId)
    print(f"========================datasetType = {data_product} ============================================")
    print("fullId..................:",ref.dataId.full)
    print("visit...................:",ref.dataId["visit"])
    print("band....................:",ref.dataId["band"])
    print("physical filter.........:",ref.dataId["physical_filter"])
    print("run.....................:",ref.run)
   
    the_visit = ref.dataId["visit"]
    the_exposure = the_visit
    the_day_obs = ref.dataId["visit"]//100_000
    the_seq_num = ref.dataId["visit"]- the_day_obs*100_000    
    the_dataId = {'day_obs': the_day_obs,'seq_num':the_seq_num,'detector':0,}
    
    # extract time
    raw_img= butler.get('raw', dataId={'exposure': the_visit, 'instrument': 'LATISS', 'detector': 0}, collections = 'LATISS/raw/all')
    meta = raw_img.getMetadata()
    md = meta.toDict()
    date_obs = md['DATE-OBS']
    
    print(the_dataId)
    df_tab =  butler.get(ref,collections=my_collection)

    if not isinstance(df_tab, pd.DataFrame):
        print("Not a pandas Dataframe")
   
   
    N=len(df_tab)
    df_tab['visid_idx']= (i*np.ones(N)).astype(int)
   
    df_tab['dateobs']= (pd.to_datetime(date_obs))
    df_tab['mjd'] = Time(date_obs).mjd
    df_tab['seq_num'] = the_seq_num
    
    all_sourcesTables.append(df_tab) 
    all_dataId.append(the_dataId) 
    all_dateobs.append(date_obs)
    #print(df_tab)
    
 
    
    #registry_query = f"instrument='LATISS' and instrument=0 and visit={the_visit}]"
    #print(f"registry_query = {registry_query}")
    #info = registry.queryDimensionRecords('exposure',where = registry_query)
    
    #if i > 1:
    #    break

========================datasetType = finalized_src_table ============================================
fullId..................: {instrument: 'LATISS', visit: 2023051100262, band: 'r', physical_filter: 'SDSSr_65mm~empty'}
visit...................: 2023051100262
band....................: r
physical filter.........: SDSSr_65mm~empty
run.....................: LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871/20240207T000014Z


/tmp/ipykernel_24978/3780976038.py:11: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId..................:",ref.dataId.full)


{'day_obs': 20230511, 'seq_num': 262, 'detector': 0}


/tmp/ipykernel_24978/3780976038.py:30: UserWarning: Collections should not be specified with DatasetRef
  df_tab =  butler.get(ref,collections=my_collection)


========================datasetType = finalized_src_table ============================================
fullId..................: {instrument: 'LATISS', visit: 2023051100263, band: 'g', physical_filter: 'SDSSg_65mm~empty'}
visit...................: 2023051100263
band....................: g
physical filter.........: SDSSg_65mm~empty
run.....................: LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871/20240207T000014Z


...

/tmp/ipykernel_24978/3780976038.py:30: UserWarning: Collections should not be specified with DatasetRef
  df_tab =  butler.get(ref,collections=my_collection)
/tmp/ipykernel_24978/3780976038.py:11: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId..................:",ref.dataId.full)


{'day_obs': 20230829, 'seq_num': 460, 'detector': 0}
========================datasetType = finalized_src_table ============================================
fullId..................: {instrument: 'LATISS', visit: 2023082900461, band: 'i', physical_filter: 'SDSSi_65mm~empty'}
visit...................: 2023082900461
band....................: i
physical filter.........: SDSSi_65mm~empty
run.....................: LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871/20240207T000014Z


/tmp/ipykernel_24978/3780976038.py:30: UserWarning: Collections should not be specified with DatasetRef
  df_tab =  butler.get(ref,collections=my_collection)
/tmp/ipykernel_24978/3780976038.py:11: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId..................:",ref.dataId.full)


{'day_obs': 20230829, 'seq_num': 461, 'detector': 0}
========================datasetType = finalized_src_table ============================================
fullId..................: {instrument: 'LATISS', visit: 2023082900462, band: 'i', physical_filter: 'SDSSi_65mm~empty'}
visit...................: 2023082900462
band....................: i
physical filter.........: SDSSi_65mm~empty
run.....................: LATISS/runs/AUXTEL_DRP_IMAGING_20230509_20240201/w_2024_05/PREOPS-4871/20240207T000014Z


/tmp/ipykernel_24978/3780976038.py:30: UserWarning: Collections should not be specified with DatasetRef
  df_tab =  butler.get(ref,collections=my_collection)
/tmp/ipykernel_24978/3780976038.py:11: FutureWarning: Call to deprecated function (or staticmethod) full. (DataCoordinate.full is deprecated in favor of .mapping, and will be dropped after v27.) -- Deprecated since version v27.
  print("fullId..................:",ref.dataId.full)


In [ ]:
ref.expanded

In [ ]:
#df_tab

In [ ]:
#t = Table.from_pandas(df_tab)

In [ ]:
len(all_sourcesTables)

In [ ]:
#all_sourcesTables[0]

In [ ]:
df=pd.concat(all_sourcesTables)

In [ ]:
df

In [ ]:
df.to_csv(filesource_out)

In [ ]:
print(df.columns.tolist())

In [ ]:
df['dateobs'].unique()